In [121]:
from random import randint, seed

def shortestVector(matrix):
    """
    Returns the norm of the shortest vector and its index.
    """
    return sorted([(matrix[idx].norm().n(), matrix[idx], idx) for idx in range(matrix.nrows())])[0]

def randomMatrix(dimension, per):
    list = [randint(-per, per) for _ in range(dimension**2)]
    M = matrix(ZZ, dimension, dimension, list)
    while M.rank() != dimension:
        list = [randint(-per, per) for _ in range(dimension**2)]
        M = matrix(ZZ, dimension, dimension, list)
    return M

def gram_matrix(matrix):
    """
    Calculate Gram matrix of given matrix.
    :param matrix: given matrix
    :return: Gram matrix of matrix
    """
    return matrix * matrix.transpose()

# def find_real_minimum_old(G, i, x):
#     """
#     Returns [y, ...] such that y[i] = x[i] and yGy^T is minimal.
#     """
#     variables = [var('y_{}'.format(j)) for j in range(G.nrows())]
#     y = deepcopy(variables)
#     y[i] = x[i]
#     Y =  matrix(1, G.nrows(), y)
#     to_minimize = (Y * G * Y.transpose())[0]
#     partials =  []
#     eqs = []
#     for variable in variables:
#         derivative = diff(to_minimize, variable)[0]
#         partials.append(derivative)
#         eqs.append(derivative == 0)
#     y.remove(y[i])
#     tmp = list(map(list, solve(eqs, y, solution_dict=True)[0].items())) # jdu blejt 
#     # https://www.geeksforgeeks.org/python-extract-dictionary-values-list-to-list/
#     # now I am missing a single y_i I removed earlier, I want to add it for clarity later
#     # absolutely horrendous code, please bear with me
#     tmp.insert(i, [variables[i], x[i]])
#     # horrendous code -- end
#     return vector([i[1].n(digits=5) for i in tmp])

def find_real_minimum(G, i, x) -> vector:
    """
    Returns [y, ...] such that y[i] = x[i] and yGy^T is minimal.
    """
    matrixA = matrix(G.nrows() - 1, G.nrows() - 1, 0)
    matrixB = matrix(G.nrows() - 1, 1, 0)
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(G.nrows()):
        if row != i:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != i]
            matrixB[b] = sum([x * G[row,j] for j in range(len(G[row])) if j == i])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right((-1)*matrixB)).list()
    result.insert(i,x)
    return vector(result).n(digits=5) 



In [128]:
def innerComparsion(dimension, perimeter, B, B_red, v_min, x, G, i):
    solutions = find_real_minimum(G, i, x[i])
    for j in range(dimension - 1):  # FIXME zbytecny forcyklus???
        difference = x[j] - solutions[j]
        if difference > 1: 
            if all([sol == 0 for sol in solutions]): # FIXME this can be simplified
                break
            return into_dict(B, G, x, solutions)
        
def matrix_to_list(A) -> list:
    # also transforms the numbers into python int
    return [[int(A.list()[row * A.ncols() + col]) for col in range(A.ncols())] for row in range(A.nrows())]

def vector_to_list(vect) -> list:
    # also transforms the numbers into python float
    return [float(num) for num in vect.list()]

def into_dict(B, G, x, solutions) -> dict:
    result = {}
    result["B"] = B
    result["G"] = G
    result["The linear combination given by LLL"] = x
    result["The linear combination suggested by cube"] = solutions
    result["The difference"] = x - vector(solutions)
    result["Shortest vector given by LLL"] = x*B
    return result


In [131]:
import json

def main():
    output_data = []
    for iteration in range(1000):
        dimension = 3
        perimeter = 10
        B = randomMatrix(dimension, perimeter)
        B_red = B.LLL()
        v_min = shortestVector(B_red)[1]
        x = B.solve_left(v_min) # find linear combination corresponding to shortest vector found by LLL
        G = gram_matrix(B)
        for i in range(G.nrows()):
            data = innerComparsion(dimension, perimeter, B, B_red, v_min, x, G, i) # terrible
            if data != None:
                output_data.append(data)
    print_the_output(output_data)
    into_json(format_data(output_data))

def format_data(output_data):
    for dicti in output_data:
        for item in dicti.items():
            if isinstance(item[1], sage.matrix.matrix_integer_dense.Matrix_integer_dense):
                dicti[item[0]] = matrix_to_list(item[1])
            else: # FIXME here I assume that all other instances in ouput_data are one-line vectors/FreeModules
                dicti[item[0]] = vector_to_list(item[1])
    return output_data

def into_json(output_data):
    out_file = open("myfile.json", "w") 
    json.dump(output_data, out_file) 
    out_file.close() 
    # FIXME https://stackoverflow.com/questions/67346184/adding-a-linebreak-after-every-element-of-python-list-in-json-file


def print_the_output(output_data) -> None:
    for dict in output_data:
        for pair in dict.items():
            print(pair[1], ": ", pair[0])
        print()
    
main()
print("Done")

[ 3 -3  2]
[-4  3 -3]
[ 3 -4  1] :  B
[ 22 -27  23]
[-27  34 -27]
[ 23 -27  26] :  G
(7, 4, -2) :  The linear combination given by LLL
(5.5789, 2.8421, -2.0000) :  The linear combination suggested by cube
(1.4211, 1.1579, 0.00000) :  The difference
(-1, -1, 0) :  Shortest vector given by LLL

[-3  0 10]
[ 4 -8  0]
[ 8 -9 -5] :  B
[109 -12 -74]
[-12  80 104]
[-74 104 170] :  G
(1, -1, 1) :  The linear combination given by LLL
(1.0000, -2.0316, 1.6782) :  The linear combination suggested by cube
(0.00000, 1.0316, -0.67816) :  The difference
(1, -1, 5) :  Shortest vector given by LLL

[ -7 -10   5]
[ -4  -3   3]
[  9   3  -3] :  B
[ 174   73 -108]
[  73   34  -54]
[-108  -54   99] :  G
(-1, 4, 1) :  The linear combination given by LLL
(-0.45997, 2.5758, 1.0000) :  The linear combination suggested by cube
(-0.54003, 1.4242, 0.00000) :  The difference
(0, 1, 4) :  Shortest vector given by LLL

Done


In [120]:
# for single matrix

H = matrix([[5,2,4],[1,3,5],[4,5,2]])
HLLL = H.LLL()
v_min = shortestVector(HLLL)[1]
v_min


(-1, 3, -2)